In [ ]:
%pylab notebook

# Evaluate the results of anisotropic training

## Fetch the eTRIMS data to compare against

In [ ]:
import urllib2
import zipfile
import os
import sys
import numpy as np
from glob import glob


def download(url, ):
    chunksize=2**20
    output_path = os.path.basename(url)
    with open(output_path, 'wb') as f:
        g = urllib2.urlopen(url) 
        total = int(g.info().getheaders("Content-Length")[0])
        downloaded = 0
        while True:
            chunk = g.read(chunksize)
            f.write(chunk)
            downloaded += len(chunk)
            sys.stdout.write('\r {:3.2%} Read {:>16} of {:<16} bytes'.format( downloaded /float(total), downloaded, total))
            sys.stdout.flush()
            if len(chunk) < chunksize:
                break
    sys.stdout.write('\n')
    return output_path

In [ ]:
ETRIMS_URL_PATTERN = 'http://www.ipb.uni-bonn.de/projects/etrims_db/downloads/etrims-db_{}.zip'
ETRIMS_URLS= [ETRIMS_URL_PATTERN.format(n) for n in 'v1','beta1', 'beta2']

# !mkdir -p etrims
# zipfile.ZipFile(download(ETRIMS_URLS[0])).extractall('etrims')
# zipfile.ZipFile(download(ETRIMS_URLS[1])).extractall('etrims')
# zipfile.ZipFile(download(ETRIMS_URLS[2])).extractall('etrims')
# !rm etrims-db_beta2.zip   
# !rm etrims-db_v1.zip
# !rm etrims-db_beta1.zip

In [ ]:
from glob import glob
etrims_images = glob('etrims/etrims-db_v1/images/08*/*.jpg')
etrims_labels = [fn.replace('/images/', '/annotations/').replace('.jpg','.png') for fn in etrims_images]
print len(etrims_images), "images from etrims"

In [ ]:
from pyfacades.util.metrics import Metrics

In [ ]:
!nvidia-smi

In [ ]:
CPU = False
import caffe
if CPU:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0)

In [ ]:
import skimage.io
from skimage.morphology import binary_dilation, binary_erosion, disk
from skimage.transform import warp, ProjectiveTransform

In [ ]:
from pyfacades.rectify import Homography as AffaraRectifier

In [ ]:
from pyfacades.util import Metrics, softmax, channels_first, channels_last

In [ ]:
from pyfacades.models.driving_12x360x480 import process_strip as segment_driving

In [ ]:
import pyfacades.models.disjoint_5_labels as segnet

In [ ]:
from pyfacades.models.disjoint_5_labels.segment import process_strip

## Put it all together 

In [ ]:
def get_metrics(f, vis=False, rectify=True):
    # Read RGB, Labels from eTRIMS
    idx = etrims_images.index(f)
    rgb = skimage.io.imread(f)
    labels = skimage.io.imread(etrims_labels[idx])
    expected_windows = (labels == array([0,0,128])).all(2).astype(int)   
    se = disk(2)
    edges = binary_dilation(expected_windows, selem=se) & ~binary_erosion(expected_windows, selem=se)
    expected_windows[edges] = 2
    
    # Rectify
    presegment = segment_driving(channels_first(rgb))
    mask = presegment.building() > 0.5
    rectifier = AffaraRectifier(rgb, mask)

    if rectify:    
        rectified_rgb = rectifier.rectified
        rectified_mask = rectifier.rectified_mask
        rectified_labels = warp(labels, ProjectiveTransform(rectifier.H), preserve_range=True).astype(np.uint8)
    else:
        rectified_rgb = rgb/255.
        rectified_mask = mask
        rectified_labels = labels
        
    rectified_windows = (rectified_labels == array([0,0,128])).all(2)

    # Predict / Inference
    probs, confs = process_strip(channels_first(rectified_rgb*255))
    rectified_prediction = probs[segnet.model.WINDOW]
    
    # Un-Rectify for comparison
    if rectify:
        prediction =  warp(rectified_prediction, ProjectiveTransform(rectifier.inv_H))
    else:
        prediction = rectified_prediction
    predicted_windows = prediction > 0.5
    #predicted_windows[~mask] = 0

    mf = Metrics(expected=expected_windows, 
             predicted=predicted_windows, 
             label_positive=1,
             label_negative=0,
             source=f,
             feature='window'
            )
    if vis:
        clf()
        subplot(321)
        imshow(rgb)
        axis('off')
        subplot(322)
        imshow(labels)
        axis('off')
        subplot(323)
        imshow(expected_windows, vmin=0, vmax=2)
        axis('off')
        subplot(324)
        imshow(predicted_windows, vmin=0, vmax=2)
        axis('off')
        subplot(325)
        rectifier.plot_rectified()
        title(None)
        axis('off')
        subplot(326)
        imshow(prediction, vmin=0, vmax=1, cmap=cm.gray)
        axis('off')

        try:
            suptitle('A:{}, P:{}, R:{}'.format(mf.pixel_accuracy, mf.pixel_precision, mf.pixel_recall))
        except ZeroDivisionError:
            pass

    return mf, channels_first(rgb), expected_windows, predicted_windows

In [ ]:
figure(figsize=(8,8))
mf, rgb, expected, predicted = get_metrics(etrims_images[12], vis=True,  rectify=True)

In [ ]:
import anydbm
import json
import hashlib
import munch

In [ ]:
eval_results = anydbm.open('eval_segnet_based_on_etrims', 'c')
checksum = hashlib.md5(open(segnet.model.WEIGHTS).read()).hexdigest()

In [ ]:
if 'md5' in eval_results and eval_results['md5'] == checksum:
    print "We already seem to have run evaluation..."

In [ ]:
len(etrims_images)

In [ ]:

recompute = False
visualize = False

if visualize:
    fig = figure(figsize=(6,6))

total = Metrics(feature='windows')
for i, f in enumerate(etrims_images):   
    if recompute or f not in eval_results:
        mf, rgb, expected, predicted = get_metrics(i, visualize)
        eval_results[f] = json.dumps(mf.as_dict())
        if visualize:
            try:
                suptitle('{} of {}, $P$:{:.2%}, $R$:{:.2%}, $F_1$:{:.2%}, $A$:{:.2%}'.format(i, len(etrims_images), total.pixel_precision, total.pixel_recall, total.pixel_f1, total.pixel_accuracy))
            except ZeroDivisionError:
                suptitle("Not enough samples yet....")         
            fig.canvas.draw()
    else:
        mf = Metrics(**json.loads(eval_results[f]))
    print '\r{} of {}'.format(i, len(etrims_images)),
    #print mf, 
    total += mf
    #print total
    
print total

# Qualitative Results

In [ ]:
%run -i color_coded_errors.py

In [ ]:
tag = 'segnet'
db = 'eTRIMS'

In [ ]:
cmp_comparison_files = [fn.strip() for fn in open('eTRIMS_comparison_files.txt')]

import os
fig = figure(figsize=(9,9))
plt.subplots_adjust(wspace=0)
for i in range(9):
    subplot(3,3,i+1)
    cached = '{}-{}-top-{}.png'.format(tag, db, i+1)
    #os.remove(cached)
    if os.path.isfile(cached):
        err_image = imread(cached)
    else:
        err_image = render_errors(cmp_comparison_files[i], alpha=0.6);
        imsave(cached, err_image)
    imshow(err_image)
    xticks([]); yticks([]);
    #xlabel(cached, fontsize=8)
    fig.canvas.draw()
fig.tight_layout()
savefig('{}-{}-top-9-figure.png'.format(tag, db), dpi=400)

In [ ]:
accuracies = [Metrics(**json.loads(eval_results[f])).pixel_recall for f in etrims_images]
argworst = argsort(accuracies)
worst = array(etrims_images)[argworst]
worst_indices = [etrims_images.index(k) for k in worst]

In [ ]:
figure()
plot(array(accuracies)[argworst]);

In [ ]:
figure(figsize=(8, 16))
result = get_metrics(worst_indices[-1])
subplot(321); title('original')
subplot(322); title('label-colors')
subplot(323); title('expected')
subplot(324); title('predicted')
subplot(325); title('rectified')
subplot(326); title('$\Pr[\text{window}]$')
savefig('worst_example.png', dpi=300)

In [ ]:
figure(figsize=(8, 16))
result = get_metrics(worst_indices[0])
subplot(321); title('original')
subplot(322); title('label-colors')
subplot(323); title('expected')
subplot(324); title('predicted')
subplot(325); title('rectified')
subplot(326); title('$\Pr[\text{window}]$')
savefig('worst_example.png', dpi=300)

In [ ]:
figure(figsize=(4, 8))
result = get_metrics(worst_indices[1])
subplot(321); title('original')
subplot(322); title('label-colors')
subplot(323); title('expected')
subplot(324); title('predicted')
subplot(325); title('rectified')
subplot(326); title('$\Pr[\text{window}]$')

In [ ]:
figure(figsize=(4, 8))
result = get_metrics(worst_indices[2])
subplot(321); title('original')
subplot(322); title('label-colors')
subplot(323); title('expected')
subplot(324); title('predicted')
subplot(325); title('rectified')
subplot(326); title('$\Pr[\text{window}]$')